In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')
import requests

# Keep only ['Office', 'Multifamily', 'Industrial', 'Mixed Use', 'Retail', 'Hospitality']
# Create sell_year: sale_year else recording_year

In [ ]:
df = pd.read_csv('part-00000-property_sales_data.csv')
df = df[df['std_land_use_code_category'].isin(['Office', 'Multifamily', 'Industrial', 'Mixed Use', 'Retail', 'Hospitality'])]
original_df = df # keeping old data for later (ctrl-f original_df to see where)
df.dropna(how='all', axis=1, inplace=True) # drop empty columns (76)
df['recording_date'] = df['recording_date'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d'))
df['recording_year'] = df['recording_date'].apply(lambda x: datetime.datetime.strftime(x, '%Y'))
df['recording_month'] = df['recording_date'].apply(lambda x: datetime.datetime.strftime(x, '%m'))
df['sale_date'] = pd.to_datetime(df['sale_date'], errors='coerce')
df['sale_year']=[datetime.datetime.strftime(x, '%Y') if x==x else x for x in df['sale_date']]
df['sale_month']=[datetime.datetime.strftime(x, '%m') if x==x else x for x in df['sale_date']]
(df['recording_date'] - df['sale_date']).describe()
df['sell_date'] = np.where(df['sale_date'].isnull(), df['recording_date'], df['sale_date'])
df['sell_year'] = df['sell_date'].apply(lambda x: datetime.datetime.strftime(x, '%Y'))
df['sell_month'] = df['sell_date'].apply(lambda x: datetime.datetime.strftime(x, '%m'))
df.head(5)

# Creating main data frame (df): will hold 1990-2019 data for each property
# => 30 years data for 151,384 properties
# 9 features for now: 'property_id','year', 'category', 'county', 'city', 'zipcode', '#times sold', '#years since last sale', 'sold?'
# Features in the future will be joined to this dataframe (df)

In [ ]:
property_ids = df['property_id'].unique()
years = [str(y) for y in list(range(1990, 2019+1))]
len(property_ids), len(years)

In [ ]:
# takes many hours to run, for testing try on just first x properties using list(enumerate(property_ids))[:x]
data = np.zeros((151384*30, 9), dtype=object)
for n,property_id in list(enumerate(property_ids)): # for each property
    temp = df[df['property_id']==property_id] # temp = sub dataframe corresponding to that property
    years_sold = list(temp['sell_year']) # all the years it was sold in (year shows up multiple times if sold multiple times that year)
    last_sold_year = sorted(years_sold)[0] # will hold the last year it was sold in, initialized as the first year it was sold in
    category = temp['std_land_use_code_category'].iloc[0]
    county = temp['fips_county'].iloc[0]
    city = temp['situs_std_city'].iloc[0]
    zipcode = temp['situs_std_zip5'].iloc[0]
    for i,year in enumerate(years): # for each year in 1990-2019 (30years)
        num_times_sold = years_sold.count(year) # how many times the property was sold that year
        if num_times_sold: last_sold_year = year # if it was sold atleast once that year => update last_sold_year to that year
        years_since_last_sold = int(year) - int(last_sold_year) # number of years since it was last sold (if it was sold this year then it's 0, if it hasn't been sold yet then it's negative -- will be fixed later)
        data[n*30+i] = [property_id, year, category, county, city, zipcode, num_times_sold, years_since_last_sold, int(num_times_sold>0)]
df = pd.DataFrame(data, columns=['property_id','year', 'category', 'county', 'city', 'zipcode', '#times sold', '#years since last sale', 'sold?'])
df.head(300)

# fill missing zipcodes using lat lon

In [ ]:
def get_zipcode(lat, lon):
    api_key = ''
    url = f'https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lon}&key={api_key}'
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print('status code error')
            return 'NOPE'
        response_json = response.json()
        results = response_json.get('results')
        for component in results[0].get('address_components'):
            if component.get('types')==['postal_code']:
                zipcode = component.get('long_name')
        print('Success', zipcode)
    except:
        print('Fail')
        return 'NOPE'
            
    return zipcode
situs_zipcodes = original_df.loc[:, ['property_id', 'situs_std_zip5' ,'lat', 'lon']].drop_duplicates()
situs_zipcodes.set_index('property_id', inplace=True)
situs_zipcodes.columns = ['zipcode', 'lat', 'lon']
situs_zipcodes['zipcode'] = situs_zipcodes.apply(lambda row: get_zipcode(row['lat'], row['lon']) if np.isnan(row['zipcode']) else row['zipcode'], axis=1)
df = df.join(situs_zipcodes.drop(['lat', 'lon'], axis=1), on='property_id')

# fill missing cities using lat lon

In [ ]:
def get_city(lat, lon):
    api_key = ''
    url = f'https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lon}&key={api_key}'
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print('status code error')
            return np.nan
        response_json = response.json()
        results = response_json.get('results')
        for result in results:
            for component in result.get('address_components'):
                if component.get('types')==['locality', 'political']:
                    city = component.get('long_name')
        print('Success', city)
    except:
        print('Fail')
        return np.nan
            
    return city
situs_city = original_df.loc[:, ['property_id', 'situs_std_city' ,'lat', 'lon']].drop_duplicates()
situs_city.set_index('property_id', inplace=True)
situs_city.columns = ['city', 'lat', 'lon']
situs_city['city'] = situs_city.apply(lambda row: get_city(row['lat'], row['lon']) if (row['city']!=row['city']) else row['city'], axis=1)
df = df.join(situs_city.drop(['lat', 'lon'], axis=1), on='property_id')

# creating features: 
'total_sales_year', 'total_sales_year_category', 'total_sales_year_county', 'total_sales_year_city', 'total_sales_year_zipcode', 'total_sales_year_category_zipcode'
'category sales/total sales', 'county sales/total sales', 'city sales/total sales', 'zipcode sales/total sales', 'category_zipcode sales/total sales'

In [ ]:
years = [str(y) for y in list(range(1990, 2019+1))]
categories = df['category'].unique()
counties = df['county'].unique()
cities = df['city'].unique()
zipcodes = df['zipcode'].unique()
# these dicts will hold the actual data for each feature, will then be converted to dataframe and joined to main df
number_of_sales_year = dict() # {year: number of sales that year}
number_of_sales_year_category = dict() # {(year,category): number of sales that year and category}
number_of_sales_year_county = dict() # {(year,county): number of sales that year and county}
number_of_sales_year_city = dict() # {(year,city): number of sales that year and city}
number_of_sales_year_zipcode = dict() # {(year,zipcode): number of sales that year and zipcode}
number_of_sales_year_cat_and_zip = dict() # {(year,category,zipcode): number of sales that year and category and zipcode}
for year in years:
    temp = df[df['year']==int(year)]
    number_of_sales_year[year] = temp['#times sold'].sum() # how many sales there were that year
    for category in categories:
        number_of_sales_year_category[(year, category)] = temp[temp['category']==category]['#times sold'].sum() # how many sales there were that year and in that category
        for zipcode in zipcodes:
            number_of_sales_year_cat_and_zip[(year, category, zipcode)] = temp[(temp['category']==category)&(temp['zipcode']==zipcode)]['#times sold'].sum() # how many sales there were that year and in that category and in that zipcode
    for county in counties:
        number_of_sales_year_county[(year, county)] = temp[temp['county']==county]['#times sold'].sum() # how many sales there were that year and in that county
    for city in cities:
        number_of_sales_year_city[(year, city)] = temp[temp['city']==city]['#times sold'].sum() # how many sales there were that year and in that city
    for zipcode in zipcodes:
        number_of_sales_year_zipcode[(year, zipcode)] = temp[temp['zipcode']==zipcode]['#times sold'].sum() # how many sales there were that year and in that zipcode
        
# joining 'total_sales_year'
total_sales_year = pd.DataFrame.from_dict(number_of_sales_year, orient='index', columns=['total_sales_year'])
total_sales_year.index = total_sales_year.index.astype(int)
df = df.join(total_sales_year, on='year')
# joining 'total_sales_year_category'
total_sales_year_category = pd.DataFrame.from_dict(number_of_sales_year_category, orient='index', columns=['total_sales_year_category'])
total_sales_year_category['index'] = total_sales_year_category.index
total_sales_year_category['year'] = total_sales_year_category['index'].apply(lambda x: int(x[0]))
total_sales_year_category['category'] = total_sales_year_category['index'].apply(lambda x: x[1])
total_sales_year_category.set_index(['year', 'category'], inplace=True)
total_sales_year_category.drop('index', axis=1, inplace=True)
df = df.join(total_sales_year_category, on=['year', 'category'])
# joining 'total_sales_year_county'
total_sales_year_county = pd.DataFrame.from_dict(number_of_sales_year_county, orient='index', columns=['total_sales_year_county'])
total_sales_year_county['index'] = total_sales_year_county.index
total_sales_year_county['year'] = total_sales_year_county['index'].apply(lambda x: int(x[0]))
total_sales_year_county['county'] = total_sales_year_county['index'].apply(lambda x: x[1])
total_sales_year_county.set_index(['year', 'county'], inplace=True)
total_sales_year_county.drop('index', axis=1, inplace=True)
df = df.join(total_sales_year_county, on=['year', 'county'])
# joining 'total_sales_year_city'
total_sales_year_city = pd.DataFrame.from_dict(number_of_sales_year_city, orient='index', columns=['total_sales_year_city'])
total_sales_year_city['index'] = total_sales_year_city.index
total_sales_year_city['year'] = total_sales_year_city['index'].apply(lambda x: int(x[0]))
total_sales_year_city['city'] = total_sales_year_city['index'].apply(lambda x: x[1])
total_sales_year_city.set_index(['year', 'city'], inplace=True)
total_sales_year_city.drop('index', axis=1, inplace=True)
df = df.join(total_sales_year_city, on=['year', 'city'])
# joining 'total_sales_year_zipcode'
total_sales_year_zipcode = pd.DataFrame.from_dict(number_of_sales_year_zipcode, orient='index', columns=['total_sales_year_zipcode'])
total_sales_year_zipcode['index'] = total_sales_year_zipcode.index
total_sales_year_zipcode['year'] = total_sales_year_zipcode['index'].apply(lambda x: int(x[0]))
total_sales_year_zipcode['zipcode'] = total_sales_year_zipcode['index'].apply(lambda x: x[1])
total_sales_year_zipcode.set_index(['year', 'zipcode'], inplace=True)
total_sales_year_zipcode.drop('index', axis=1, inplace=True)
df = df.join(total_sales_year_zipcode, on=['year', 'zipcode'])
# joining 'total_sales_year_category_zipcode'
total_sales_year_category_zipcode = pd.DataFrame.from_dict(number_of_sales_year_cat_and_zip, orient='index', columns=['total_sales_year_category_zipcode'])
total_sales_year_category_zipcode['index'] = total_sales_year_category_zipcode.index
total_sales_year_category_zipcode['year'] = total_sales_year_category_zipcode['index'].apply(lambda x: int(x[0]))
total_sales_year_category_zipcode['category'] = total_sales_year_category_zipcode['index'].apply(lambda x: x[1])
total_sales_year_category_zipcode['zipcode'] = total_sales_year_category_zipcode['index'].apply(lambda x: x[2])
total_sales_year_category_zipcode.set_index(['year', 'category', 'zipcode'], inplace=True)
total_sales_year_category_zipcode.drop('index', axis=1, inplace=True)
df = df.join(total_sales_year_category_zipcode, on=['year', 'category','zipcode'])

# creating the ratio features
df['category sales/total sales'] = df['total_sales_year_category']/df['total_sales_year']
df['county sales/total sales'] = df['total_sales_year_county']/df['total_sales_year']
df['city sales/total sales'] = df['total_sales_year_city']/df['total_sales_year']
df['zipcode sales/total sales'] = df['total_sales_year_zipcode']/df['total_sales_year']
df['category_zipcode sales/total sales'] = df['total_sales_year_category_zipcode']/df['total_sales_year']

# sqft features

In [ ]:
sqft = original_df[['property_id', 'std_land_use_code_category','lot_size_sqft']].drop_duplicates()
# outlier treatment per category: values below 1% quantile -> 1% quantile, 95% quantile <- values above 95% quantile
# missing value treatment: fill na's by avg sqft in its category
for category in categories:
    upperbound = sqft[sqft['std_land_use_code_category'] == category]['lot_size_sqft'].quantile(0.95)
    lowerbound = sqft[sqft['std_land_use_code_category'] == category]['lot_size_sqft'].quantile(0.01)
    sqft['lot_size_sqft'] = np.where(((sqft['std_land_use_code_category'] == category) & (sqft['lot_size_sqft'] >= upperbound)), upperbound, sqft['lot_size_sqft'])
    sqft['lot_size_sqft'] = np.where(((sqft['std_land_use_code_category'] == category) & (sqft['lot_size_sqft'] <= lowerbound)), lowerbound, sqft['lot_size_sqft'])
    avg = sqft[sqft['std_land_use_code_category']==category]['lot_size_sqft'].mean()
    sqft[sqft['std_land_use_code_category']==category]=sqft[sqft['std_land_use_code_category']==category].fillna(avg)
# join
sqft = sqft[['property_id', 'lot_size_sqft']]
sqft.set_index('property_id', inplace=True)
df = df.join(sqft, on='property_id')
#
total_sqft_year = dict()
total_sqft_year_category = dict()
total_sqft_year_county = dict()
total_sqft_year_city = dict()
total_sqft_year_zipcode = dict()
total_sqft_year_cat_and_zip = dict()
for year in years:
    temp = df[(df['year']==int(year))&(df['sold?']==1)]
    total_sqft_year[year] = sum(temp['lot_size_sqft']*temp['#times sold']) # total sqft of properties sold that year
    for category in categories:
        total_sqft_year_category[(year, category)] = sum(temp[temp['category']==category]['lot_size_sqft']*temp[temp['category']==category]['#times sold']) # total sqft of properties sold that year and in that category
        for zipcode in zipcodes:
            total_sqft_year_cat_and_zip[(year, category, zipcode)] = sum(temp[(temp['category']==category)&(temp['zipcode']==zipcode)]['lot_size_sqft']*temp[(temp['category']==category)&(temp['zipcode']==zipcode)]['#times sold']) # total sqft of properties sold that year and in that category and in that zipcode
    for county in counties:
        total_sqft_year_county[(year, county)] = sum(temp[temp['county']==county]['lot_size_sqft']*temp[temp['county']==county]['#times sold']) # total sqft of properties sold that year and in that county
    for city in cities:
        total_sqft_year_city[(year, city)] = sum(temp[temp['city']==city]['lot_size_sqft']*temp[temp['city']==city]['#times sold']) # total sqft of properties sold that year and in that city
    for zipcode in zipcodes:
        total_sqft_year_zipcode[(year, zipcode)] = sum(temp[temp['zipcode']==zipcode]['lot_size_sqft']*temp[temp['zipcode']==zipcode]['#times sold']) # total sqft of properties sold that year and in that zipcode
# join
df_total_sqft_year = pd.DataFrame.from_dict(total_sqft_year, orient='index', columns=['total_sqft_year'])
df_total_sqft_year.index = df_total_sqft_year.index.astype(int)
df = df.join(df_total_sqft_year, on='year')
#
df_total_sqft_year_category = pd.DataFrame.from_dict(total_sqft_year_category, orient='index', columns=['total_sqft_year_category'])
df_total_sqft_year_category['index'] = df_total_sqft_year_category.index
df_total_sqft_year_category['year'] = df_total_sqft_year_category['index'].apply(lambda x: int(x[0]))
df_total_sqft_year_category['category'] = df_total_sqft_year_category['index'].apply(lambda x: x[1])
df_total_sqft_year_category.set_index(['year', 'category'], inplace=True)
df_total_sqft_year_category.drop('index', axis=1, inplace=True)
df = df.join(df_total_sqft_year_category, on=['year', 'category'])
#
df_total_sqft_year_county = pd.DataFrame.from_dict(total_sqft_year_county, orient='index', columns=['total_sqft_year_county'])
df_total_sqft_year_county['index'] = df_total_sqft_year_county.index
df_total_sqft_year_county['year'] = df_total_sqft_year_county['index'].apply(lambda x: int(x[0]))
df_total_sqft_year_county['county'] = df_total_sqft_year_county['index'].apply(lambda x: x[1])
df_total_sqft_year_county.set_index(['year', 'county'], inplace=True)
df_total_sqft_year_county.drop('index', axis=1, inplace=True)
df = df.join(df_total_sqft_year_county, on=['year', 'county'])
#
df_total_sqft_year_city = pd.DataFrame.from_dict(total_sqft_year_city, orient='index', columns=['total_sqft_year_city'])
df_total_sqft_year_city['index'] = df_total_sqft_year_city.index
df_total_sqft_year_city['year'] = df_total_sqft_year_city['index'].apply(lambda x: int(x[0]))
df_total_sqft_year_city['city'] = df_total_sqft_year_city['index'].apply(lambda x: x[1])
df_total_sqft_year_city.set_index(['year', 'city'], inplace=True)
df_total_sqft_year_city.drop('index', axis=1, inplace=True)
df = df.join(df_total_sqft_year_city, on=['year', 'city'])
#
df_total_sqft_year_zipcode = pd.DataFrame.from_dict(total_sqft_year_zipcode, orient='index', columns=['total_sqft_year_zipcode'])
df_total_sqft_year_zipcode['index'] = df_total_sqft_year_zipcode.index
df_total_sqft_year_zipcode['year'] = df_total_sqft_year_zipcode['index'].apply(lambda x: int(x[0]))
df_total_sqft_year_zipcode['zipcode'] = df_total_sqft_year_zipcode['index'].apply(lambda x: x[1])
df_total_sqft_year_zipcode.set_index(['year', 'zipcode'], inplace=True)
df_total_sqft_year_zipcode.drop('index', axis=1, inplace=True)
df = df.join(df_total_sqft_year_zipcode, on=['year', 'zipcode'])
#
df_total_sqft_year_cat_and_zipcode = pd.DataFrame.from_dict(total_sqft_year_cat_and_zip, orient='index', columns=['total_sqft_year_category_zipcode'])
df_total_sqft_year_cat_and_zipcode['index'] = df_total_sqft_year_cat_and_zipcode.index
df_total_sqft_year_cat_and_zipcode['year'] = df_total_sqft_year_cat_and_zipcode['index'].apply(lambda x: int(x[0]))
df_total_sqft_year_cat_and_zipcode['category'] = df_total_sqft_year_cat_and_zipcode['index'].apply(lambda x: x[1])
df_total_sqft_year_cat_and_zipcode['zipcode'] = df_total_sqft_year_cat_and_zipcode['index'].apply(lambda x: x[2])
df_total_sqft_year_cat_and_zipcode.set_index(['year', 'category', 'zipcode'], inplace=True)
df_total_sqft_year_cat_and_zipcode.drop('index', axis=1, inplace=True)
df = df.join(df_total_sqft_year_cat_and_zipcode, on=['year', 'category','zipcode'])
#
df['avg_sqft_year'] = df['total_sqft_year'] / df['total_sales_year']
df['avg_sqft_year_category'] = df['total_sqft_year_category'] / df['total_sales_year_category']
df['avg_sqft_year_county'] = df['total_sqft_year_county'] / df['total_sales_year_county']
df['avg_sqft_year_city'] = df['total_sqft_year_city'] / df['total_sales_year_city']
df['avg_sqft_year_zipcode'] = df['total_sqft_year_zipcode'] / df['total_sales_year_zipcode']
df['avg_sqft_year_category_zipcode'] = df['total_sqft_year_category_zipcode'] / df['total_sales_year_category_zipcode']
#
df['sqft/avg_sqft_year'] = df['lot_size_sqft'] / df['avg_sqft_year']
df['sqft/avg_sqft_year_category'] = df['lot_size_sqft'] / df['avg_sqft_year_category']
df['sqft/avg_sqft_year_county'] = df['lot_size_sqft'] / df['avg_sqft_year_county']
df['sqft/avg_sqft_year_city'] = df['lot_size_sqft'] / df['avg_sqft_year_city']
df['sqft/avg_sqft_year_zipcode'] = df['lot_size_sqft'] / df['avg_sqft_year_zipcode']
df['sqft/avg_sqft_year_category_zipcode'] = df['lot_size_sqft'] / df['avg_sqft_year_category_zipcode']

# nrooms features

In [ ]:
nrooms = df[['property_id','category','lot_size_sqft']].drop_duplicates()
temp = original_df[['property_id','sum_number_of_rooms']].drop_duplicates()
temp.set_index('property_id', inplace=True)
nrooms = nrooms.join(temp, on='property_id')
# avg sqft per category
category_avg = {}
for category in categories:
    avg = nrooms[nrooms['category']==category]['lot_size_sqft'].mean()
    category_avg[category] = avg
avg_sqft = pd.DataFrame.from_dict(category_avg, orient='index', columns=['avg_sqft'])
nrooms = nrooms.join(avg_sqft, on='category')
# avg nrooms per category
nrooms_avg = {}
for category in categories:
    avg = nrooms[nrooms['category']==category]['sum_number_of_rooms'].mean()
    nrooms_avg[category] = avg
avg_nrooms = pd.DataFrame.from_dict(nrooms_avg, orient='index', columns=['avg_nrooms'])
nrooms = nrooms.join(avg_nrooms, on='category')
# missing value treatment: fill na with mean_nrooms_category*(sqft/avg_sqft_category),
# i.e fill na with average nrooms in its category scaled by how big the property is relative to the average size in its category
nrooms['nrooms'] = np.where(nrooms['sum_number_of_rooms'].isna(), round(nrooms['avg_nrooms']*nrooms['lot_size_sqft']/nrooms['avg_sqft']), nrooms['sum_number_of_rooms'])
nrooms = nrooms[['property_id', 'nrooms']]
nrooms.set_index('property_id', inplace=True)
df = df.join(nrooms, on='property_id')
#
total_nrooms_year = dict()
total_nrooms_year_category = dict()
total_nrooms_year_county = dict()
total_nrooms_year_city = dict()
total_nrooms_year_zipcode = dict()
total_nrooms_year_category_and_zip = dict()
for year in years:
    temp = df[(df['year']==int(year))&(df['sold?']==1)]
    total_nrooms_year[year] = sum(temp['nrooms']*temp['#times sold'])
    for category in categories:
        total_nrooms_year_category[(year, category)] = sum(temp[temp['category']==category]['nrooms']*temp[temp['category']==category]['#times sold'])
        for zipcode in zipcodes:
            total_nrooms_year_category_and_zip[(year, category, zipcode)] = sum(temp[(temp['category']==category)&(temp['zipcode']==zipcode)]['nrooms']*temp[(temp['category']==category)&(temp['zipcode']==zipcode)]['#times sold'])
    for county in counties:
        total_nrooms_year_county[(year, county)] = sum(temp[temp['county']==county]['nrooms']*temp[temp['county']==county]['#times sold'])
    for city in cities:
        total_nrooms_year_city[(year, city)] = sum(temp[temp['city']==city]['nrooms']*temp[temp['city']==city]['#times sold'])
    for zipcode in zipcodes:
        total_nrooms_year_zipcode[(year, zipcode)] = sum(temp[temp['zipcode']==zipcode]['nrooms']*temp[temp['zipcode']==zipcode]['#times sold'])
# join
df_total_nrooms_year = pd.DataFrame.from_dict(total_nrooms_year, orient='index', columns=['total_nrooms_year'])
df_total_nrooms_year.index = df_total_nrooms_year.index.astype(int)
df = df.join(df_total_nrooms_year, on='year')
#
df_total_nrooms_year_category = pd.DataFrame.from_dict(total_nrooms_year_category, orient='index', columns=['total_nrooms_year_category'])
df_total_nrooms_year_category['index'] = df_total_nrooms_year_category.index
df_total_nrooms_year_category['year'] = df_total_nrooms_year_category['index'].apply(lambda x: int(x[0]))
df_total_nrooms_year_category['category'] = df_total_nrooms_year_category['index'].apply(lambda x: x[1])
df_total_nrooms_year_category.set_index(['year', 'category'], inplace=True)
df_total_nrooms_year_category.drop('index', axis=1, inplace=True)
df = df.join(df_total_nrooms_year_category, on=['year', 'category'])
#
df_total_nrooms_year_county = pd.DataFrame.from_dict(total_nrooms_year_county, orient='index', columns=['total_nrooms_year_county'])
df_total_nrooms_year_county['index'] = df_total_nrooms_year_county.index
df_total_nrooms_year_county['year'] = df_total_nrooms_year_county['index'].apply(lambda x: int(x[0]))
df_total_nrooms_year_county['county'] = df_total_nrooms_year_county['index'].apply(lambda x: x[1])
df_total_nrooms_year_county.set_index(['year', 'county'], inplace=True)
df_total_nrooms_year_county.drop('index', axis=1, inplace=True)
df = df.join(df_total_nrooms_year_county, on=['year', 'county'])
#
df_total_nrooms_year_city = pd.DataFrame.from_dict(total_nrooms_year_city, orient='index', columns=['total_nrooms_year_city'])
df_total_nrooms_year_city['index'] = df_total_nrooms_year_city.index
df_total_nrooms_year_city['year'] = df_total_nrooms_year_city['index'].apply(lambda x: int(x[0]))
df_total_nrooms_year_city['city'] = df_total_nrooms_year_city['index'].apply(lambda x: x[1])
df_total_nrooms_year_city.set_index(['year', 'city'], inplace=True)
df_total_nrooms_year_city.drop('index', axis=1, inplace=True)
df = df.join(df_total_nrooms_year_city, on=['year', 'city'])
#
df_total_nrooms_year_zipcode = pd.DataFrame.from_dict(total_nrooms_year_zipcode, orient='index', columns=['total_nrooms_year_zipcode'])
df_total_nrooms_year_zipcode['index'] = df_total_nrooms_year_zipcode.index
df_total_nrooms_year_zipcode['year'] = df_total_nrooms_year_zipcode['index'].apply(lambda x: int(x[0]))
df_total_nrooms_year_zipcode['zipcode'] = df_total_nrooms_year_zipcode['index'].apply(lambda x: x[1])
df_total_nrooms_year_zipcode.set_index(['year', 'zipcode'], inplace=True)
df_total_nrooms_year_zipcode.drop('index', axis=1, inplace=True)
df = df.join(df_total_nrooms_year_zipcode, on=['year', 'zipcode'])
#
df_total_nrooms_year_category_and_zip = pd.DataFrame.from_dict(total_nrooms_year_category_and_zip, orient='index', columns=['total_nrooms_year_category_zipcode'])
df_total_nrooms_year_category_and_zip['index'] = df_total_nrooms_year_category_and_zip.index
df_total_nrooms_year_category_and_zip['year'] = df_total_nrooms_year_category_and_zip['index'].apply(lambda x: int(x[0]))
df_total_nrooms_year_category_and_zip['category'] = df_total_nrooms_year_category_and_zip['index'].apply(lambda x: x[1])
df_total_nrooms_year_category_and_zip['zipcode'] = df_total_nrooms_year_category_and_zip['index'].apply(lambda x: x[2])
df_total_nrooms_year_category_and_zip.set_index(['year', 'category', 'zipcode'], inplace=True)
df_total_nrooms_year_category_and_zip.drop('index', axis=1, inplace=True)
df = df.join(df_total_nrooms_year_category_and_zip, on=['year', 'category','zipcode'])
#
df['avg_nrooms_year'] = df['total_nrooms_year'] / df['total_sales_year']
df['avg_nrooms_year_category'] = df['total_nrooms_year_category'] / df['total_sales_year_category']
df['avg_nrooms_year_county'] = df['total_nrooms_year_county'] / df['total_sales_year_county']
df['avg_nrooms_year_city'] = df['total_nrooms_year_city'] / df['total_sales_year_city']
df['avg_nrooms_year_zipcode'] = df['total_nrooms_year_zipcode'] / df['total_sales_year_zipcode']
df['avg_nrooms_year_category_zipcode'] = df['total_nrooms_year_category_zipcode'] / df['total_sales_year_category_zipcode']
#
df['nrooms/avg_nrooms_year'] = df['nrooms'] / df['avg_nrooms_year']
df['nrooms/avg_nrooms_year_category'] = df['nrooms'] / df['avg_nrooms_year_category']
df['nrooms/avg_nrooms_year_county'] = df['nrooms'] / df['avg_nrooms_year_county']
df['nrooms/avg_nrooms_year_city'] = df['nrooms'] / df['avg_nrooms_year_city']
df['nrooms/avg_nrooms_year_zipcode'] = df['nrooms'] / df['avg_nrooms_year_zipcode']
df['nrooms/avg_nrooms_year_category_zipcode'] = df['nrooms'] / df['avg_nrooms_year_category_zipcode']

# nbaths & nbedrooms features

In [ ]:
nbaths = df[['property_id','category','lot_size_sqft']].drop_duplicates()
temp = original_df[['property_id','total_baths']].drop_duplicates()
temp.set_index('property_id', inplace=True)
nbaths = nbaths.join(temp, on='property_id')
# avg sqft per category
category_avg = {}
for category in categories:
    avg = nbaths[nbaths['category']==category]['lot_size_sqft'].mean()
    category_avg[category]=avg
avg_sqft = pd.DataFrame.from_dict(category_avg, orient='index', columns=['avg_sqft'])
nbaths = nbaths.join(avg_sqft, on='category')
# avg nbaths per category
nbaths_avg = {}
for category in categories:
    avg = nbaths[nbaths['category']==category]['total_baths'].mean()
    nbaths_avg[category]=avg
avg_nbaths = pd.DataFrame.from_dict(nbaths_avg, orient='index', columns=['avg_nbaths'])
nbaths = nbaths.join(avg_nbaths, on='category')
# missing value treatment: fill na with avg_nbaths_category*(sqft/avg_sqft_category),
# i.e fill na with average nbaths in its category scaled by how big the property is relative to the average size in its category
nbaths['nbaths'] = np.where(nbaths['total_baths'].isna(), round(nbaths['avg_nbaths']*nbaths['lot_size_sqft']/nbaths['avg_sqft']), nbaths['total_baths'])
nbaths = nbaths[['property_id', 'nbaths']]
nbaths.set_index('property_id', inplace=True)
df = df.join(nbaths, on='property_id')
#########################################################################################################
# nbedrooms
nbedrooms = df[['property_id','category','lot_size_sqft']].drop_duplicates()
temp = original_df[['property_id','sum_number_of_bedrooms']].drop_duplicates()
temp.set_index('property_id', inplace=True)
nbedrooms = nbedrooms.join(temp, on='property_id')
# avg sqft per category
category_avg = {}
for category in categories:
    avg = nbedrooms[nbedrooms['category']==category]['lot_size_sqft'].mean()
    category_avg[category]=avg
avg_sqft = pd.DataFrame.from_dict(category_avg, orient='index', columns=['avg_sqft'])
nbedrooms = nbedrooms.join(avg_sqft, on='category')
# avg nbedrooms per category
nbedrooms_avg = {}
for category in categories:
    avg = nbedrooms[nbedrooms['category']==category]['sum_number_of_bedrooms'].mean()
    nbedrooms_avg[category]=avg
avg_nbedrooms = pd.DataFrame.from_dict(nbedrooms_avg, orient='index', columns=['avg_nbedrooms'])
nbedrooms = nbedrooms.join(avg_nbedrooms, on='category')
# missing value treatment: fill na with avg_nbedrooms_category*(sqft/avg_sqft_category),
# i.e fill na with average nbedrooms in its category scaled by how big the property is relative to the average size in its category
nbedrooms['nbedrooms'] = np.where(nbedrooms['sum_number_of_bedrooms'].isna(), round(nbedrooms['avg_nbedrooms']*nbedrooms['lot_size_sqft']/nbedrooms['avg_sqft']), nbedrooms['sum_number_of_bedrooms'])
nbedrooms = nbedrooms[['property_id', 'nbedrooms']]
nbedrooms.set_index('property_id', inplace=True)
df = df.join(nbedrooms, on='property_id')
#
total_nbaths_year = dict()
total_nbaths_year_category = dict()
total_nbaths_year_county = dict()
total_nbaths_year_city = dict()
total_nbaths_year_zipcode = dict()
total_nbaths_year_category_zipcode = dict()
#
total_nbedrooms_year = dict()
total_nbedrooms_year_category = dict()
total_nbedrooms_year_county = dict()
total_nbedrooms_year_city = dict()
total_nbedrooms_year_zipcode = dict()
total_nbedrooms_year_category_zipcode = dict()
for year in years:
    temp = df[(df['year']==int(year))&(df['sold?']==1)]
    total_nbedrooms_year[year] = sum(temp['nbedrooms']*temp['#times sold'])
    total_nbaths_year[year] = sum(temp['nbaths']*temp['#times sold'])
    for category in categories:
        total_nbedrooms_year_category[(year, category)] = sum(temp[temp['category']==category]['nbedrooms']*temp[temp['category']==category]['#times sold'])
        total_nbaths_year_category[(year, category)] = sum(temp[temp['category']==category]['nbaths']*temp[temp['category']==category]['#times sold'])
        for zipcode in zipcodes:
            total_nbedrooms_year_category_zipcode[(year, category, zipcode)] = sum(temp[(temp['category']==category)&(temp['zipcode']==zipcode)]['nbedrooms']*temp[(temp['category']==category)&(temp['zipcode']==zipcode)]['#times sold'])
            total_nbaths_year_category_zipcode[(year, category, zipcode)] = sum(temp[(temp['category']==category)&(temp['zipcode']==zipcode)]['nbaths']*temp[(temp['category']==category)&(temp['zipcode']==zipcode)]['#times sold'])
    for county in counties:
        total_nbedrooms_year_county[(year, county)] = sum(temp[temp['county']==county]['nbedrooms']*temp[temp['county']==county]['#times sold'])
        total_nbaths_year_county[(year, county)] = sum(temp[temp['county']==county]['nbaths']*temp[temp['county']==county]['#times sold'])
    for city in cities:
        total_nbedrooms_year_city[(year, city)] = sum(temp[temp['city']==city]['nbedrooms']*temp[temp['city']==city]['#times sold'])
        total_nbaths_year_city[(year, city)] = sum(temp[temp['city']==city]['nbaths']*temp[temp['city']==city]['#times sold'])
    for zipcode in zipcodes:
        total_nbedrooms_year_zipcode[(year, zipcode)] = sum(temp[temp['zipcode']==zipcode]['nbedrooms']*temp[temp['zipcode']==zipcode]['#times sold'])
        total_nbaths_year_zipcode[(year, zipcode)] = sum(temp[temp['zipcode']==zipcode]['nbaths']*temp[temp['zipcode']==zipcode]['#times sold'])
# join
df_total_nbedrooms_year = pd.DataFrame.from_dict(total_nbedrooms_year, orient='index', columns=['total_nbedrooms_year'])
df_total_nbedrooms_year.index = df_total_nbedrooms_year.index.astype(int)
df = df.join(df_total_nbedrooms_year, on='year')
#
df_total_nbedrooms_year_category = pd.DataFrame.from_dict(total_nbedrooms_year_category, orient='index', columns=['total_nbedrooms_year_category'])
df_total_nbedrooms_year_category['index'] = df_total_nbedrooms_year_category.index
df_total_nbedrooms_year_category['year'] = df_total_nbedrooms_year_category['index'].apply(lambda x: int(x[0]))
df_total_nbedrooms_year_category['category'] = df_total_nbedrooms_year_category['index'].apply(lambda x: x[1])
df_total_nbedrooms_year_category.set_index(['year', 'category'], inplace=True)
df_total_nbedrooms_year_category.drop('index', axis=1, inplace=True)
df = df.join(df_total_nbedrooms_year_category, on=['year', 'category'])
#
df_total_nbedrooms_year_county = pd.DataFrame.from_dict(total_nbedrooms_year_county, orient='index', columns=['total_nbedrooms_year_county'])
df_total_nbedrooms_year_county['index'] = df_total_nbedrooms_year_county.index
df_total_nbedrooms_year_county['year'] = df_total_nbedrooms_year_county['index'].apply(lambda x: int(x[0]))
df_total_nbedrooms_year_county['county'] = df_total_nbedrooms_year_county['index'].apply(lambda x: x[1])
df_total_nbedrooms_year_county.set_index(['year', 'county'], inplace=True)
df_total_nbedrooms_year_county.drop('index', axis=1, inplace=True)
df = df.join(df_total_nbedrooms_year_county, on=['year', 'county'])
#
df_total_nbedrooms_year_city = pd.DataFrame.from_dict(total_nbedrooms_year_city, orient='index', columns=['total_nbedrooms_year_city'])
df_total_nbedrooms_year_city['index'] = df_total_nbedrooms_year_city.index
df_total_nbedrooms_year_city['year'] = df_total_nbedrooms_year_city['index'].apply(lambda x: int(x[0]))
df_total_nbedrooms_year_city['city'] = df_total_nbedrooms_year_city['index'].apply(lambda x: x[1])
df_total_nbedrooms_year_city.set_index(['year', 'city'], inplace=True)
df_total_nbedrooms_year_city.drop('index', axis=1, inplace=True)
df = df.join(df_total_nbedrooms_year_city, on=['year', 'city'])
#
df_total_nbedrooms_year_zipcode = pd.DataFrame.from_dict(total_nbedrooms_year_zipcode, orient='index', columns=['total_nbedrooms_year_zipcode'])
df_total_nbedrooms_year_zipcode['index'] = df_total_nbedrooms_year_zipcode.index
df_total_nbedrooms_year_zipcode['year'] = df_total_nbedrooms_year_zipcode['index'].apply(lambda x: int(x[0]))
df_total_nbedrooms_year_zipcode['zipcode'] = df_total_nbedrooms_year_zipcode['index'].apply(lambda x: x[1])
df_total_nbedrooms_year_zipcode.set_index(['year', 'zipcode'], inplace=True)
df_total_nbedrooms_year_zipcode.drop('index', axis=1, inplace=True)
df = df.join(df_total_nbedrooms_year_zipcode, on=['year', 'zipcode'])
#
df_total_nbedrooms_year_category_zipcode = pd.DataFrame.from_dict(total_nbedrooms_year_category_zipcode, orient='index', columns=['total_nbedrooms_year_category_zipcode'])
df_total_nbedrooms_year_category_zipcode['index'] = df_total_nbedrooms_year_category_zipcode.index
df_total_nbedrooms_year_category_zipcode['year'] = df_total_nbedrooms_year_category_zipcode['index'].apply(lambda x: int(x[0]))
df_total_nbedrooms_year_category_zipcode['category'] = df_total_nbedrooms_year_category_zipcode['index'].apply(lambda x: x[1])
df_total_nbedrooms_year_category_zipcode['zipcode'] = df_total_nbedrooms_year_category_zipcode['index'].apply(lambda x: x[2])
df_total_nbedrooms_year_category_zipcode.set_index(['year', 'category', 'zipcode'], inplace=True)
df_total_nbedrooms_year_category_zipcode.drop('index', axis=1, inplace=True)
df = df.join(df_total_nbedrooms_year_category_zipcode, on=['year', 'category','zipcode'])
#
feature = 'nbedrooms'
df[f'avg_{feature}_year'] = df[f'total_{feature}_year'] / df['total_sales_year']
df[f'avg_{feature}_year_category'] = df[f'total_{feature}_year_category'] / df['total_sales_year_category']
df[f'avg_{feature}_year_county'] = df[f'total_{feature}_year_county'] / df['total_sales_year_county']
df[f'avg_{feature}_year_city'] = df[f'total_{feature}_year_city'] / df['total_sales_year_city']
df[f'avg_{feature}_year_zipcode'] = df[f'total_{feature}_year_zipcode'] / df['total_sales_year_zipcode']
df[f'avg_{feature}_year_category_zipcode'] = df[f'total_{feature}_year_category_zipcode'] / df['total_sales_year_category_zipcode']

df[f'{feature}/avg_{feature}_year'] = df[f'{feature}'] / df[f'avg_{feature}_year']
df[f'{feature}/avg_{feature}_year_category'] = df[f'{feature}'] / df[f'avg_{feature}_year_category']
df[f'{feature}/avg_{feature}_year_county'] = df[f'{feature}'] / df[f'avg_{feature}_year_county']
df[f'{feature}/avg_{feature}_year_city'] = df[f'{feature}'] / df[f'avg_{feature}_year_city']
df[f'{feature}/avg_{feature}_year_zipcode'] = df[f'{feature}'] / df[f'avg_{feature}_year_zipcode']
df[f'{feature}/avg_{feature}_year_category_zipcode'] = df[f'{feature}'] / df[f'avg_{feature}_year_category_zipcode']
#
df_total_nbaths_year = pd.DataFrame.from_dict(total_nbaths_year, orient='index', columns=['total_nbaths_year'])
df_total_nbaths_year.index = df_total_nbaths_year.index.astype(int)
df = df.join(df_total_nbaths_year, on='year')
#
df_total_nbaths_year_category = pd.DataFrame.from_dict(total_nbaths_year_category, orient='index', columns=['total_nbaths_year_category'])
df_total_nbaths_year_category['index'] = df_total_nbaths_year_category.index
df_total_nbaths_year_category['year'] = df_total_nbaths_year_category['index'].apply(lambda x: int(x[0]))
df_total_nbaths_year_category['category'] = df_total_nbaths_year_category['index'].apply(lambda x: x[1])
df_total_nbaths_year_category.set_index(['year', 'category'], inplace=True)
df_total_nbaths_year_category.drop('index', axis=1, inplace=True)
df = df.join(df_total_nbaths_year_category, on=['year', 'category'])
#
df_total_nbaths_year_county = pd.DataFrame.from_dict(total_nbaths_year_county, orient='index', columns=['total_nbaths_year_county'])
df_total_nbaths_year_county['index'] = df_total_nbaths_year_county.index
df_total_nbaths_year_county['year'] = df_total_nbaths_year_county['index'].apply(lambda x: int(x[0]))
df_total_nbaths_year_county['county'] = df_total_nbaths_year_county['index'].apply(lambda x: x[1])
df_total_nbaths_year_county.set_index(['year', 'county'], inplace=True)
df_total_nbaths_year_county.drop('index', axis=1, inplace=True)
df = df.join(df_total_nbaths_year_county, on=['year', 'county'])
#
df_total_nbaths_year_city = pd.DataFrame.from_dict(total_nbaths_year_city, orient='index', columns=['total_nbaths_year_city'])
df_total_nbaths_year_city['index'] = df_total_nbaths_year_city.index
df_total_nbaths_year_city['year'] = df_total_nbaths_year_city['index'].apply(lambda x: int(x[0]))
df_total_nbaths_year_city['city'] = df_total_nbaths_year_city['index'].apply(lambda x: x[1])
df_total_nbaths_year_city.set_index(['year', 'city'], inplace=True)
df_total_nbaths_year_city.drop('index', axis=1, inplace=True)
df = df.join(df_total_nbaths_year_city, on=['year', 'city'])
#
df_total_nbaths_year_zipcode = pd.DataFrame.from_dict(total_nbaths_year_zipcode, orient='index', columns=['total_nbaths_year_zipcode'])
df_total_nbaths_year_zipcode['index'] = df_total_nbaths_year_zipcode.index
df_total_nbaths_year_zipcode['year'] = df_total_nbaths_year_zipcode['index'].apply(lambda x: int(x[0]))
df_total_nbaths_year_zipcode['zipcode'] = df_total_nbaths_year_zipcode['index'].apply(lambda x: x[1])
df_total_nbaths_year_zipcode.set_index(['year', 'zipcode'], inplace=True)
df_total_nbaths_year_zipcode.drop('index', axis=1, inplace=True)
df = df.join(df_total_nbaths_year_zipcode, on=['year', 'zipcode'])
#
df_total_nbaths_year_category_zipcode = pd.DataFrame.from_dict(total_nbaths_year_category_zipcode, orient='index', columns=['total_nbaths_year_category_zipcode'])
df_total_nbaths_year_category_zipcode['index'] = df_total_nbaths_year_category_zipcode.index
df_total_nbaths_year_category_zipcode['year'] = df_total_nbaths_year_category_zipcode['index'].apply(lambda x: int(x[0]))
df_total_nbaths_year_category_zipcode['category'] = df_total_nbaths_year_category_zipcode['index'].apply(lambda x: x[1])
df_total_nbaths_year_category_zipcode['zipcode'] = df_total_nbaths_year_category_zipcode['index'].apply(lambda x: x[2])
df_total_nbaths_year_category_zipcode.set_index(['year', 'category', 'zipcode'], inplace=True)
df_total_nbaths_year_category_zipcode.drop('index', axis=1, inplace=True)
df = df.join(df_total_nbaths_year_category_zipcode, on=['year', 'category','zipcode'])
#
feature = 'nbaths'
df[f'avg_{feature}_year'] = df[f'total_{feature}_year'] / df['total_sales_year']
df[f'avg_{feature}_year_category'] = df[f'total_{feature}_year_category'] / df['total_sales_year_category']
df[f'avg_{feature}_year_county'] = df[f'total_{feature}_year_county'] / df['total_sales_year_county']
df[f'avg_{feature}_year_city'] = df[f'total_{feature}_year_city'] / df['total_sales_year_city']
df[f'avg_{feature}_year_zipcode'] = df[f'total_{feature}_year_zipcode'] / df['total_sales_year_zipcode']
df[f'avg_{feature}_year_category_zipcode'] = df[f'total_{feature}_year_category_zipcode'] / df['total_sales_year_category_zipcode']

df[f'{feature}/avg_{feature}_year'] = df[f'{feature}'] / df[f'avg_{feature}_year']
df[f'{feature}/avg_{feature}_year_category'] = df[f'{feature}'] / df[f'avg_{feature}_year_category']
df[f'{feature}/avg_{feature}_year_county'] = df[f'{feature}'] / df[f'avg_{feature}_year_county']
df[f'{feature}/avg_{feature}_year_city'] = df[f'{feature}'] / df[f'avg_{feature}_year_city']
df[f'{feature}/avg_{feature}_year_zipcode'] = df[f'{feature}'] / df[f'avg_{feature}_year_zipcode']
df[f'{feature}/avg_{feature}_year_category_zipcode'] = df[f'{feature}'] / df[f'avg_{feature}_year_category_zipcode']

# new feature: age

In [ ]:
year_built = original_df.groupby(['property_id', 'year_built']).size().reset_index()[['property_id','year_built']].set_index('property_id')
df = df.join(year_built, on='property_id')
year_first_sold = original_df.groupby(['property_id', 'sell_year']).size().reset_index().drop_duplicates('property_id', keep='first')[['property_id', 'sell_year']].set_index('property_id')
year_first_sold.columns = ['year_first_sold']
df = df.join(year_first_sold, on='property_id')
df.year_built = np.where(df.year_built.isna(), df.year_first_sold, df.year_built)
df['age'] = df['year'] - df['year_built'].apply(int)

# new feature: category_sales/number_properties_category

In [ ]:
# create category_sales / number of properties in that category (constant denominator throughout the years)
new_feature = pd.DataFrame(df.groupby(['year', 'category'])['#times sold'].sum()).join(pd.DataFrame(df.groupby('category').size(), columns=['n_properties_category']))
new_feature['category_sales/properties_category'] = new_feature['#times sold']/new_feature['n_properties_category']
new_feature.drop(['#times sold', 'n_properties_category'], axis=1, inplace=True)
df = df.join(new_feature, on=['year', 'category'])
del new_feature

# correct years since last sale to -1 if negative

In [ ]:
df['#years since last sale'] = np.where(df['#years since last sale']<0, -1, df['#years since last sale'])

# df => Lagging data 10 years => new_df

In [ ]:
# features we want to lagg & how many laggs
lagg_features = ['#years since last sale', '#times sold', 'category_sales/properties_category',
                 'total_sales_year_category', 'total_sales_year_zipcode', 'total_sales_year_category_zipcode',
                 'category_zipcode sales/total sales', 'zipcode sales/total sales',
                'sqft/avg_sqft_year_category_zipcode', 'nrooms/avg_nrooms_year_category_zipcode', 'nbedrooms/avg_nbedrooms_year_category_zipcode',
                'nbaths/avg_nbaths_year_category_zipcode']
lag = 9 # lag+1 number of laggs

In [ ]:
new_dict = {}
new_dict['property_id'] = df['property_id']
new_dict['category'] = df['category']
new_dict['county'] = df['county']
new_dict['city'] = df['city']
new_dict['zipcode'] = df['zipcode']
new_dict['nrooms'] = df['nrooms']
new_dict['nbedrooms'] = df['nbedrooms']
new_dict['nbaths'] = df['nbaths']
new_dict['sqft'] = df['lot_size_sqft']
new_dict['year'] = df['year']
for col_name in lagg_features:
    new_dict[col_name] = df[col_name]
    for l in range(1, lag+1):
        new_dict['%s_t-%d' %(col_name,l)] = df.groupby('property_id')[col_name].shift(l)
new_df = pd.DataFrame(new_dict, index=df.index)
new_df.dropna(inplace=True)
new_df['target_year'] = new_df['year'] + 1
new_df = new_df.join(df[['property_id','year', 'sold?']].set_index(['property_id', 'year']), on=['property_id', 'target_year'])
new_df['sold_t+1'] = new_df['sold?']
new_df.drop('sold?', axis=1, inplace=True)

In [ ]:
#new_df[~new_df['sold_t+1'].isna()].to_csv('new_df_10.csv', index=False) # data to train/validate/test

In [ ]:
#new_df[new_df['sold_t+1'].isna()].to_csv('new_df_10_2020.csv', index=False) # data for 2020 pred (sold_t+1 is na ofcourse)